# Stack Exchange Project - by Natalie Mazal

In [1]:
import pandas as pd

In [2]:
comments=pd.read_csv('c:/tmp/comments.csv')

In [3]:
posts=pd.read_csv('c:/tmp/posts.csv')

In [4]:
users=pd.read_csv('c:/tmp/users.csv')

In [5]:
votes=pd.read_csv('c:/tmp/votes.csv')

# Basic Analysis

1. how many posts were made each year?

In [20]:
posts['CreationDate']=pd.to_datetime(posts['CreationDate'])
posts['creation_year']=posts['CreationDate'].dt.year
posts.groupby('creation_year')['Id'].count().to_frame().rename(columns={'Id':'num_of_posts'})

,num_of_posts
creation_year,
2011,156
2012,1065
2013,1018


2. how many votes were made in each day of the week ?

In [21]:

import calendar
def dayname(daynum):
    return (calendar.day_name[daynum])

votes['CreationDate']=pd.to_datetime(votes['CreationDate'])
votes['day_week_creation']=votes['CreationDate'].dt.dayofweek.apply(dayname)
votes.groupby('day_week_creation')['Id'].count()\
.to_frame().rename(columns={'Id':'num_of_votes'})\
.sort_values('num_of_votes',ascending=False)


,num_of_votes
day_week_creation,
Thursday,869
Friday,735
Wednesday,554
Saturday,376
Tuesday,354
Monday,282
Sunday,189


3. list all comments created on September 19th,2012

In [22]:
mask=comments['CreationDate'].str.contains("2012-09-19")
comments[mask].head()

,Id,PostId,Text,CreationDate,CommentUserId
1414,3476,887,I had the same thoughts. I'm wondering if ther...,2012-09-19 10:34:12.983,233
1484,3623,890,"hmmm he's described as ""an over-the-top public...",2012-09-19 18:23:37.193,126


4.list all users under the age of 33,living in London

In [23]:
mask=(users['Age']<33)&(users['Location'].str.contains('London'))

users[mask].head()

,UserId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,Age,AboutMe,Views,UpVotes,DownVotes
27,27,101,2011-11-30 18:58:00.620,Noldorin,2011-12-27 00:54:12.993,http://noldorin.com/,"London, United Kingdom",24.0,<p>entrepreneur; graduate in mathematics / the...,0,0,0
155,160,101,2011-12-04 14:01:19.123,Willbill,2013-02-06 10:19:52.963,NaN,"London, United Kingdom",31.0,NaN,1,6,0
451,473,101,2011-12-29 16:57:11.727,bobble14988,2011-12-29 16:57:11.727,http://www.bobbleworks.co.uk,"London, United Kingdom",26.0,<p>Third year university student who recently ...,0,0,0
679,710,101,2012-01-26 14:06:04.333,wim,2014-09-01 16:18:32.173,http://www.google.com,"London, United Kingdom",32.0,"<p>Hi from London, England! I hope my answers...",0,7,0
686,717,101,2012-01-27 17:26:03.490,JHarris,2012-02-09 14:22:38.973,http://www.johnharris.tv,"London, UK",29.0,NaN,1,0,0


# Advanced Analysis

1. display the number of votes for each post title

In [24]:
df=votes.merge(posts,how='inner',left_on='PostId',right_on='Id')
df_merge=df.groupby('Title')['Id_x'].count().to_frame()\
.rename(columns={'Id_x':'num_of_votes'})\
.sort_values('num_of_votes',ascending=False)
df_merge.head()

,num_of_votes
Title,
Have there been other candidates to play Jack Reacher?,27
Was Disney's The Black Hole the last film to feature an overture?,26
Where did the speaking-disease come from?,26
"In The Departed, did the FBI know about the mole in the police department and did they interfere with the investigation?",25
Why does Harry only make use of time travel in The Prisoner of Azkaban?,23


2. display posts with comments created by users living in the same location a the post creator.

In [25]:
#step1 : join users and posts
users.rename(columns={'Id':'UserId'},inplace=True)
posts_merge_users=posts.merge(users,how='inner',left_on='OwnerUserId',right_on='UserId')\
.rename(columns={'Location':'PostCreatedLocation','OwnerUserId':'PostCreatedId','Id':'PostId'})\
[['PostCreatedId','Title','PostCreatedLocation','PostId']]
#step2 : add info on Comment Created Location
comments.rename(columns={'UserId':'CommentUserId'},inplace=True)
comments_merge_users=comments.merge(users,how='inner',left_on='CommentUserId',right_on='UserId')\
.rename(columns={'Location':'CommentCreatedLocation'})\
[['CommentCreatedLocation','PostId','CommentUserId']]
comments_merge_users
#step3 : join of 2 tables created above 
df=posts_merge_users.merge(comments_merge_users,how='inner',left_on='PostId',right_on='PostId')\
[['PostId','Title','CommentUserId','PostCreatedId','CommentCreatedLocation','PostCreatedLocation']]
#step4 : add mask
mask=df['CommentCreatedLocation']==df['PostCreatedLocation']
df[mask]

,PostId,Title,CommentUserId,PostCreatedId,CommentCreatedLocation,PostCreatedLocation
531,819,Was Monk's mother also OCD?,52,52,Missouri,Missouri
761,1139,Clarification about closing a loop in The Looper,293,1632,Sweden,Sweden
911,952,Thriller about a group of kidnappers wearing S...,1312,1312,"Sydney, Australia","Sydney, Australia"
946,1440,What is the Traitor's Motive?,535,1590,Canada,Canada
1420,1239,Is there more to Gandalf and Galadriel's relat...,1221,835,Sweden,Sweden


3. how many users have never voted

In [26]:
votes.rename(columns={'UserId':'VUserId'},inplace=True)
users_merge_votes=users.merge(votes,how='left',left_on='UserId',right_on='VUserId')\
[['UserId','VUserId']]
mask=users_merge_votes['VUserId'].isnull()
df=users_merge_votes[mask]['VUserId']
len(df)

355

4.display all posts having the highest amount of comments

In [27]:
comments_merge_posts=comments.merge(posts,how='left',left_on='PostId',right_on='Id')\
[['Title','Id_x']].rename(columns={'Id_x':'CommentsCount'})
dfcomments_merge_posts=comments_merge_posts.groupby('Title')['CommentsCount'].count()\
.to_frame().sort_values('CommentsCount',ascending=False)
dfcomments_merge_posts['rank_comments']=dfcomments_merge_posts['CommentsCount'].rank(ascending=False,method='dense')
mask=dfcomments_merge_posts['rank_comments']==1
dfcomments_merge_posts[mask]

,CommentsCount,rank_comments
Title,,
"How much money did it take to run ""The Sting""?",5,1.0
Old movie about two pilots and a cheating wife,5,1.0
What is Jerusalem worth?,5,1.0
How does the budget breakdown for a US TV season compare to a UK TV Season?,5,1.0
How do the 'frozen goods' survive the banana stand fire?,5,1.0
Who nukes the USA in The Divide?,5,1.0
Aliens took over the satellite,5,1.0
Why doesn’t Shosanna try to kill Fredrick?,5,1.0
"In Iron Man, why is Tony Stark fooled for so long?",5,1.0


5. For each post, how many votes are coming from users living in Canada?
   What’s their percentage of the total number of votes

In [28]:
#step1 : join posts and votes
posts_merge_votes=posts.merge(votes,how='inner',left_on='Id',right_on='PostId')\
[['Title','VUserId']]
#step2 : add info on users location 
users_posts_votes=posts_merge_votes.merge(users,how='left',left_on='VUserId',right_on='UserId')\
[['Title','VUserId','Location']]
#step3 : filter NaN values
mask=~users_posts_votes['Location'].isnull()
users_posts_votes=users_posts_votes[mask]
#step4 : count worldwide votes
world_wide_votes=users_posts_votes.groupby('Title')['VUserId'].count().to_frame().rename(columns={'VUserId':'worldwide_votes'})
#step5 : count canada votes
canada=users_posts_votes['Location'].str.contains('Canada')
canada_votes=users_posts_votes[canada]
canada_votes=canada_votes.groupby('Title')['Location'].count().to_frame().rename(columns={'Location':'Canada_Votes'})
#step6 : join worldwide cound and canada count
wwv_canada=world_wide_votes.merge(canada_votes,how='left',left_on='Title',right_on='Title')
wwv_canada['Canada_Votes'].fillna(value=0,inplace=True)
#step7 : add info on percentage 
wwv_canada['percentage']=wwv_canada['Canada_Votes']/wwv_canada['worldwide_votes']*100
wwv_canada.head()

,worldwide_votes,Canada_Votes,percentage
Title,,,
"""Road"" picture about guys in a tank, possibly set in the Middle East",4,1.0,25.000000
"""Rock TV"" animated/live action series from the early to mid 80s",3,0.0,0.000000
"""The Club"" where Jerry Seinfeld performed",1,0.0,0.000000
"""There are two kinds of people"" quote",2,0.0,0.000000
'80s Aquaman tv-series?,6,1.0,16.666667


6. How many hours in average, it takes to the first comment to be posted after a creation of a new post

In [30]:
import numpy as np
first_comment=comments.groupby('PostId')['CreationDate'].min().to_frame()\
.rename(columns={'CreationDate':'comment_creation_date'})
comments_posts=first_comment.merge(posts,how='inner',left_on='PostId',right_on='Id')\
[['comment_creation_date','CreationDate']]
comments_posts['comment_creation_date']=pd.to_datetime(comments_posts['comment_creation_date'])
comments_posts['diff']=(comments_posts['comment_creation_date']-comments_posts['CreationDate'])/np.timedelta64(1,'h')
np.round(comments_posts['diff'].mean())

7.0

7. Whats the most common post tag?

In [37]:
# split the tag name  
p=list(posts['Tags'].str.split('><'))
data=[]
for i in p:
    for item in i:
         data.append(item)
# remove '<' and '>'          
data=pd.Series(data).str.replace('<','').str.replace('>','').to_frame()
# count per tag name 
data.columns=['tag_name']
data.groupby('tag_name')['tag_name'].count().sort_values(ascending=False).head(1)

tag_name
plot-explanation    665
Name: tag_name, dtype: int64

8. Create a pivot table displaying how many posts were created for each
   year (Y axis) and each month (X axis)

In [38]:
posts["year_create"] = posts["CreationDate"].dt.year
posts["month_create"] = posts["CreationDate"].dt.month
posts_each_y_m = posts.pivot_table(index = "year_create", columns = "month_create" ,\
                                   values = "Title", aggfunc = "count")
posts_each_y_m.head()

month_create,1,2,3,4,5,6,7,8,9,10,11,12
year_create,,,,,,,,,,,,
2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,15.0,140.0
2012,92.0,72.0,62.0,88.0,93.0,105.0,105.0,81.0,60.0,75.0,77.0,155.0
2013,129.0,99.0,107.0,92.0,130.0,120.0,134.0,143.0,64.0,NaN,NaN,NaN
